In [ ]:
import xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
from tqdm import tqdm
from preprocess.sza import solarzenithangle
from utils.etc import benchmark
from dask.distributed import Client
import shutil, gc
import os

In [ ]:
client = Client(n_workers=32)

In [ ]:
stations = xarray.open_dataset('/capstor/scratch/cscs/kschuurm/DATA/DWD/netcdf/DWD_SOLAR_index.nc')
stations

In [ ]:
hrseviri = xarray.open_zarr('/capstor/scratch/cscs/kschuurm/ZARR/SEVIRI_FULLDISK.zarr')

proj = ccrs.PlateCarree()

fig, axis = plt.subplots(1, 1, subplot_kw=dict(projection=proj))

gl = axis.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')

axis.coastlines()

lat = stations.lat.values
lon = stations.lon.values
name = stations.name.values
axis.scatter(x=lon, y=lat, c='r', s=.5)
# for i, txt in enumerate(name):
#     axis.annotate(txt, (lon[i], lat[i]))

hrseviri.channel_data.sel(channel='VIS006').isel(time = 2000).plot.imshow(ax=axis, transform = proj)



In [ ]:


def create_collocated_zarr_dwd(hrseviri, station_id, patch_size=15):
    
    station = xarray.open_dataset(f'/capstor/scratch/cscs/kschuurm/DATA/DWD/netcdf/DWD_SOLAR_10min_{str(station_id).zfill(5)}.nc')
    station = station.drop_duplicates('time')

    lat = station.lat.values
    lon = station.lon.values
    ilat = (np.abs(hrseviri.y - lat)).argmin().values
    ilon = (np.abs(hrseviri.x - lon)).argmin().values

    hpatch = int(np.floor(patch_size/2))

    station['time'] = station.time - np.timedelta64(10, 'm')

    intersec = set(station.time.values).intersection(hrseviri.time.values)

    if len(intersec)< 1000:
        print(f'skipping station, too little points {station_id}')
        return None

    
    with benchmark('load'):
        hres_slice = hrseviri.isel(y=slice(ilat-hpatch, ilat + hpatch +1), x=slice(ilon-hpatch, ilon+hpatch+1)).load()
    for v in list(hres_slice.coords.keys()):
        if hres_slice.coords[v].dtype == object:
            hres_slice[v].encoding.clear()
    for var in hres_slice:
        hres_slice[var].encoding.clear()

    

    intersec_time = np.unique(np.array(list(set(station.time.values).intersection(set(hres_slice.time.values)))))
    if len(intersec_time)<50:
        print(f'skipping {station_id} to little matching timeindices')
        return None
    subset_station = station.sel(time=intersec_time)
    hres_slice = hres_slice.sel(time=intersec_time)

    hres_slice['GHI'] = ('time', (subset_station.GS_10.data/(10*60))*10000)# J/cm2 per 10 min to watt/m2
    
    hres_slice['DIF'] = ('time', (subset_station.DS_10.data/(10*60))*10000)
    hres_slice['station_name'] = str(subset_station.station_name.values)
    hres_slice['lat_station'] = subset_station.lat.data
    hres_slice['lon_station'] = subset_station.lon.data
    hres_slice['altitude_station'] = subset_station.elevation.data
    
    with benchmark('dropna'):
        hres_slice = hres_slice.dropna('time')

    with benchmark('SZA'):
        SZA, AZI = solarzenithangle(pd.to_datetime(hres_slice.time), 
                                    hres_slice.lat_station.values, 
                                    hres_slice.lon_station.values, 
                                    hres_slice.altitude_station.values)

    hres_slice['SZA'] = ('time', SZA.astype(np.float32))
    hres_slice['AZI'] = ('time', AZI.astype(np.float32))

    for v in list(hres_slice.coords.keys()):
        if hres_slice.coords[v].dtype == object:
            hres_slice[v].encoding.clear()

    hres_slice = hres_slice.chunk({'time':-1, 'channel':-1, 'y':-1, 'x':-1})
    hres_slice.attrs = station.attrs


    with benchmark('zarr'):
        hres_slice.to_zarr(SAVE_PATH + f'DWD_SOLAR_{str(station_id).zfill(5)}.zarr', mode='w')
    
    print(f'        STATION {station_id} DONE         ')

def create_collocated_zarr_knmi(hrseviri, station_id, patch_size=15):
    station = xarray.open_dataset(f'/capstor/scratch/cscs/kschuurm/DATA/KNMI/nc/KNMI_SOLAR_10min_{str(station_id)}.nc')
    station = station.drop_duplicates('time')
    print(station)
    lat = station.lat.values
    lon = station.lat.values
    ilat = (np.abs(hrseviri.y - lat)).argmin().values
    ilon = (np.abs(hrseviri.x - lon)).argmin().values

    hpatch = int(np.floor(patch_size/2))

    station['time'] = station.time - np.timedelta64(10, 'm')
    
    intersec = set(station.time.values).intersection(hrseviri.time.values)

    if len(intersec)< 1000:
        print(f'skipping station {station_id}, too little points')
        return None


    hres_slice = hrseviri.isel(y=slice(ilat-hpatch, ilat + hpatch +1), x=slice(ilon-hpatch, ilon+hpatch+1)).load()
    for v in list(hres_slice.coords.keys()):
        if hres_slice.coords[v].dtype == object:
            hres_slice[v].encoding.clear()
    for var in hres_slice:
        hres_slice[var].encoding.clear()

    intersec_time = np.sort(np.array(list(set(station.time.values).intersection(set(hres_slice.time.values)))))
    if len(intersec_time)<50:
        print(f'skipping {station_id} to little matching timeindices')
        return None
    subset_station = station.sel(time=intersec_time)
    hres_slice = hres_slice.sel(time=intersec_time)
    
    hres_slice['GHI'] = ('time', subset_station.Q_GLOB_10.data)
    hres_slice['GHI_min'] = ('time', subset_station.QN_GLOB_10.data)
    hres_slice['GHI_max'] = ('time', subset_station.QX_GLOB_10.data)
    hres_slice['station_name'] = str(station_id)
    hres_slice['lat_station'] = subset_station.lat.data
    hres_slice['lon_station'] = subset_station.lon.data
    hres_slice['altitude_station'] = subset_station.altitude.data
    
    with benchmark('dropna'):
        hres_slice = hres_slice.dropna('time')
    
    with benchmark('SZA'):
        SZA, AZI = solarzenithangle(pd.to_datetime(hres_slice.time), 
                                    hres_slice.lat_station.values, 
                                    hres_slice.lon_station.values, 
                                    hres_slice.altitude_station.values)

    hres_slice['SZA'] = ('time', SZA.astype(np.float32))
    hres_slice['AZI'] = ('time', AZI.astype(np.float32))

    for v in list(hres_slice.coords.keys()):
        if hres_slice.coords[v].dtype == object:
            hres_slice[v].encoding.clear()

    hres_slice = hres_slice.chunk({'time':-1, 'channel':-1, 'y':-1, 'x':-1})
    hres_slice.attrs = station.attrs

    with benchmark('zarr'):
        hres_slice.to_zarr(SAVE_PATH + f'KNMI_SOLAR_{str(station_id)}.zarr', mode='w')
    
    print(f'        STATION {station_id} DONE         ')
    


In [ ]:
import time
for i in tqdm(range(40)):
    time.sleep(60)

In [ ]:

SAVE_PATH = f'/capstor/scratch/cscs/kschuurm/ZARR/DWD/'
stations_dwd = xarray.open_dataset('/capstor/scratch/cscs/kschuurm/DATA/DWD/netcdf/DWD_SOLAR_index.nc')


hrseviri = xarray.open_zarr('/capstor/scratch/cscs/kschuurm/ZARR/SEVIRI_FULLDISK_timechunked.zarr')

for nm in tqdm(stations_dwd.station_id.values):
    if os.path.exists(SAVE_PATH + f'DWD_SOLAR_{str(nm).zfill(5)}.zarr'):
        print(f'skip {nm}')
        continue
    create_collocated_zarr_dwd(hrseviri, nm)



In [ ]:
SAVE_PATH = f'/capstor/scratch/cscs/kschuurm/ZARR/KNMI/'

stations_knmi = xarray.open_dataset('/capstor/scratch/cscs/kschuurm/DATA/KNMI/nc/index.nc')


    
for nm in tqdm(stations_knmi.station.values):
    
    if os.path.exists(f'/capstor/scratch/cscs/kschuurm/DATA/KNMI/nc/KNMI_SOLAR_10min_{str(nm)}.nc'):
        create_collocated_zarr_knmi(hrseviri, nm)
    else:
        print('station does not have data')
      


In [ ]:
def create_collocated_zarr_meteoswiss(hrseviri, stations_meteoswiss, station_name, patch_size=15):
    station = stations_meteoswiss.sel(station_name=station_name).load().copy(deep=True)
    
    lat = station.y.values
    lon = station.x.values
    ilat = (np.abs(hrseviri.y - lat)).argmin().values
    ilon = (np.abs(hrseviri.x - lon)).argmin().values

    hpatch = int(np.floor(patch_size/2))
    
    station['time'] = station.time - np.timedelta64(10, 'm')
    
    with benchmark('dropna station'):
        station = station.dropna('time', subset=['ssi'])
    
    intersec = set(station.time.values).intersection(hrseviri.time.values)

    if len(intersec)< 1000:
        print(f'skipping station, too little points {station_name}')
        return None


    hres_slice = hrseviri.isel(y=slice(ilat-hpatch, ilat + hpatch +1), x=slice(ilon-hpatch, ilon+hpatch+1)).load()
    for v in list(hres_slice.coords.keys()):
        if hres_slice.coords[v].dtype == object:
            hres_slice[v].encoding.clear()
    for var in hres_slice:
        hres_slice[var].encoding.clear()

        
    intersec_time = np.sort(np.array(list(set(station.time.values).intersection(set(hres_slice.time.values)))))
    if len(intersec_time)<50:
        print(f'skipping {station_name} to little matching timeindices')
        return None
    subset_station = station.sel(time=intersec_time)
    hres_slice = hres_slice.sel(time=intersec_time)
    
    hres_slice['GHI'] = ('time', subset_station.ssi.data)
    hres_slice['station_name'] = str(station_name)
    hres_slice['lat_station'] = subset_station.y.data
    hres_slice['lon_station'] = subset_station.x.data
    hres_slice['altitude_station'] = subset_station.altitude.data
    

    with benchmark('SZA'):
        SZA, AZI = solarzenithangle(pd.to_datetime(hres_slice.time), 
                                    hres_slice.lat_station.values, 
                                    hres_slice.lon_station.values, 
                                    float(hres_slice.altitude_station.values))

    hres_slice['SZA'] = ('time', SZA.astype(np.float32))
    hres_slice['AZI'] = ('time', AZI.astype(np.float32))

    for v in list(hres_slice.coords.keys()):
        if hres_slice.coords[v].dtype == object:
            hres_slice[v].encoding.clear()

    hres_slice = hres_slice.rename_dims({'y':'lat','x':'lon'}).rename({'y':'lat', 'x':'lon'})
    hres_slice = hres_slice.chunk({'time':10000, 'channel':-1, 'lat':-1, 'lon':-1})

    with benchmark('zarr'):
        hres_slice.to_zarr(SAVE_PATH + f'METEOSWISS_SOLAR_{str(station_name)}.zarr', mode='w')
    
    print(f'        STATION {station_name} DONE         ')

In [ ]:
SAVE_PATH = f'/scratch/snx3000/kschuurm/ZARR/METEOSWISS/'


stations_meteoswiss = xarray.open_zarr('/scratch/snx3000/kschuurm/DATA/METEOSWISS/ground_station_data.zarr')


hrseviri = xarray.open_zarr('/scratch/snx3000/kschuurm/ZARR/SEVIRI_FULLDISK_timeseries.zarr')


from multiprocessing.pool import ThreadPool
from functools import partial

f = partial(create_collocated_zarr_meteoswiss, hrseviri, stations_meteoswiss)

nm_not_done = []
for nm in tqdm(stations_meteoswiss.station_name.values):
    # if os.path.exists(SAVE_PATH + f'METEOSWISS_SOLAR_{nm}.zarr'):
    #     print(f'skip {nm}')
    #     continue
        
    nm_not_done.append(nm)
    
with ThreadPool(5) as pool:
    a = pool.map(f, nm_not_done)
    
    for x in tqdm(a):
        x.result()


# PLAYGROUND


In [ ]:
hrseviri = xarray.open_zarr('/capstor/scratch/cscs/kschuurm/ZARR/SEVIRI_FULLDISK_timechunked.zarr')

station = xarray.open_dataset(f'/capstor/scratch/cscs/kschuurm/DATA/DWD/netcdf/DWD_SOLAR_10min_{str(3287).zfill(5)}.nc')

lat = station.lat.values
lon = station.lon.values
ilat = (np.abs(hrseviri.y - lat)).argmin().values
ilon = (np.abs(hrseviri.x - lon)).argmin().values

hpatch = int(np.floor(15/2))



<xarray.Dataset>
Dimensions:           (time: 115431, channel: 11, lat: 15, lon: 15)
Coordinates:
  * channel           (channel) object 'IR_016' 'IR_039' ... 'WV_062' 'WV_073'
    crs               <U740 ...
  * lat               (lat) float64 46.17 46.23 46.28 ... 46.78 46.83 46.88
  * lon               (lon) float64 5.975 6.025 6.075 ... 6.575 6.625 6.675
    spatial_ref       int64 ...
  * time              (time) datetime64[ns] 2016-01-01 ... 2022-08-05T23:30:00
Data variables:
    AZI               (time) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    GHI               (time) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    SZA               (time) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    altitude_station  <U3 ...
    channel_data      (channel, time, lat, lon) float16 dask.array<chunksize=(11, 10000, 15, 15), meta=np.ndarray>
    lat_station       float32 ...
    lon_station       float32 ...
    station_name      <U3 ...

In [ ]:
station['time'] = station.time - np.timedelta64(10, 'm')

intersec = set(station.time.values).intersection(hrseviri.time.values)

# if len(intersec)< 1000:
#     print(f'skipping station, too little points {station_id}')
#     return None


with benchmark('load'):
    hres_slice = hrseviri.isel(y=slice(ilat-hpatch, ilat + hpatch +1), x=slice(ilon-hpatch, ilon+hpatch+1)).load()
for v in list(hres_slice.coords.keys()):
    if hres_slice.coords[v].dtype == object:
        hres_slice[v].encoding.clear()
for var in hres_slice:
    hres_slice[var].encoding.clear()

    
intersec_time = np.unique(np.array(list(set(station.time.values).intersection(set(hres_slice.time.values)))))

print(len(intersec_time), len(station.time))

In [ ]:
subset_station = station.sel(time=intersec_time)
hres_slice = hres_slice.sel(time=intersec_time)

hres_slice['GHI'] = ('time', (subset_station.GS_10.data/(10*60))*10000)# J/cm2 per 10 min to watt/m2

hres_slice['DIF'] = ('time', (subset_station.DS_10.data/(10*60))*10000)
hres_slice['station_name'] = str(subset_station.station_name.values)
hres_slice['lat_station'] = subset_station.lat.data
hres_slice['lon_station'] = subset_station.lon.data
hres_slice['altitude_station'] = subset_station.elevation.data

with benchmark('dropna'):
    print(len(hres_slice.time))
    hres_slice = hres_slice.dropna('time')
    print(len(hres_slice.time))

with benchmark('SZA'):
    SZA, AZI = solarzenithangle(pd.to_datetime(hres_slice.time), 
                                hres_slice.lat_station.values, 
                                hres_slice.lon_station.values, 
                                hres_slice.altitude_station.values)

hres_slice['SZA'] = ('time', SZA.astype(np.float32))
hres_slice['AZI'] = ('time', AZI.astype(np.float32))

for v in list(hres_slice.coords.keys()):
    if hres_slice.coords[v].dtype == object:
        hres_slice[v].encoding.clear()

for v in hres_slice:
    hres_slice[v].encoding.clear()
        
# hres_slice = hres_slice.rename_dims({'y':'lat','x':'lon'}).rename({'y':'lat', 'x':'lon'})
hres_slice = hres_slice.chunk({'time':10000, 'channel':-1, 'y':-1, 'x':-1})
hres_slice.attrs = station.attrs

# with benchmark('zarr'):
#     hres_slice.to_netcdf(SAVE_PATH + f'DWD_SOLAR_{str(station_id).zfill(5)}.nc', mode='w')

In [ ]:
for v in hres_slice:
    hres_slice[v].encoding.clear()
hres_slice.to_zarr(f'/capstor/scratch/cscs/kschuurm/ZARR/DWD_SOLAR_{str(3287).zfill(5)}.zarr')

In [ ]:
stations_knmi = xarray.open_dataset('/scratch/snx3000/kschuurm/DATA/KNMI/nc/index.nc')
stations_knmi

In [ ]:
station_id = stations_knmi.station[5].values
station = xarray.open_dataset(f'/scratch/snx3000/kschuurm/DATA/KNMI/nc/KNMI_SOLAR_10min_{str(station_id)}.nc')
station

In [ ]:

station = xarray.open_dataset(f'/scratch/snx3000/kschuurm/DATA/KNMI/nc/KNMI_SOLAR_10min_{str(station_id)}.nc')

lat = station.lat.values
lon = station.lon.values
ilat = (np.abs(hrseviri.y - lat)).argmin().values
ilon = (np.abs(hrseviri.x - lon)).argmin().values

patch_size = 31
hpatch = int(np.floor(patch_size/2))

station['time'] = station.time - np.timedelta64(10, 'm')

intersec = set(station.time.values).intersection(hrseviri.time.values)

if len(intersec)< 1000:
    print('skipping station')


hres_slice = hrseviri.isel(y=slice(ilat-hpatch, ilat + hpatch +1), x=slice(ilon-hpatch, ilon+hpatch+1))
for v in list(hres_slice.coords.keys()):
    if hres_slice.coords[v].dtype == object:
        hres_slice[v].encoding.clear()
for var in hres_slice:
    hres_slice[var].encoding.clear()

with benchmark('temp'):
    hres_slice.chunk({'time':10000, 'x':-1, 'y':-1}).to_zarr('temp2.zarr', mode='w')
    temp = xarray.open_zarr('temp2.zarr')
    
hres_slice = temp.where(temp.time.isin(station.time), drop=True)
_, index = np.unique(hres_slice['time'], return_index=True)
subset_station = station.isel(time=index)

hres_slice['GHI'] = ('time', subset_station.Q_GLOB_10.data)
hres_slice['GHI_min'] = ('time', subset_station.QN_GLOB_10.data)
hres_slice['GHI_max'] = ('time', subset_station.QX_GLOB_10.data)
hres_slice['station_name'] = str(station_id)
hres_slice['lat_station'] = subset_station.lat.data
hres_slice['lon_station'] = subset_station.lon.data
hres_slice['altitude_station'] = subset_station.altitude.data

with benchmark('SZA'):
    SZA, AZI = solarzenithangle(pd.to_datetime(hres_slice.time), 
                                hres_slice.lat_station.values, 
                                hres_slice.lon_station.values, 
                                hres_slice.altitude_station.values)

hres_slice['SZA'] = ('time', SZA.astype(np.float32))
hres_slice['AZI'] = ('time', AZI.astype(np.float32))

for v in list(hres_slice.coords.keys()):
    if hres_slice.coords[v].dtype == object:
        hres_slice[v].encoding.clear()

hres_slice = hres_slice.rename_dims({'y':'lat','x':'lon'}).rename({'y':'lat', 'x':'lon'})


hres_slice

In [ ]:
timeidxnotnan_seviri = np.load('/scratch/snx3000/kschuurm/ZARR/idxnotnan_seviri.npy')
timeidxnotnan_seviri


In [ ]:
len(hrseviri.time) - len(timeidxnotnan_seviri)